In [18]:
from decouple import config
from pymongo import MongoClient
import numpy as np
import pandas as pd

In [3]:
connection_string = config('MONGO_CONNECTION_STRING')

In [4]:
client = MongoClient(connection_string)
db = client.db
coll = db['twitter']

In [5]:
# all users
user_pipeline = [
    {
        "$project": {
            "_id": 0,
            "id": "$user.id",
            "name": "$user.name",
            "followers": "$user.followers_count",
            "verified": "$user.verified"
        }
    },
    {
        "$sort": {
            "name": 1
        }
    }
]

In [6]:
user_df = pd.DataFrame.from_dict(list(coll.aggregate(user_pipeline)))
user_df.head()

,id,name,followers,verified
0,807034969902252032,"""MRH_1984"" 🇨🇦",2480,False
1,942083635808792576,#AnitasAffordableBookstore,529,False
2,942083635808792576,#AnitasAffordableBookstore,529,False
3,942083635808792576,#AnitasAffordableBookstore,529,False
4,942083635808792576,#AnitasAffordableBookstore,529,False


In [7]:
user_df.drop_duplicates(subset=['id'], keep='first', inplace=True)
user_df.head()

,id,name,followers,verified
0,807034969902252032,"""MRH_1984"" 🇨🇦",2480,False
1,942083635808792576,#AnitasAffordableBookstore,529,False
13,1207754603673972736,#BCPoliTalk,206,False
15,2518034430,#FreeChelseaManning & #PrayForAmazonas,163,False
16,38458897,#IndianStatus531,1228,False


In [22]:
user_df.to_json('users.json', orient='records')

In [6]:
group_pipeline = [
    {
      "$match": {
          "is_quote_status": {"$eq": True},
          "quoted_status": {"$exists": True}
      }  
    },
    {
        "$project": {
            "_id": 0,
            "twid": "$id",
            "user_id": "$user.id",
            "user": "$user.name",
            "verified": "$user.verified",
            "created_at": {
                "$dateFromString": {
                    "dateString": "$created_at"
                }
            },
            "qu_twid": "$quoted_status.id_str",
            "qu_user_id": "$quoted_status.user.id_str",
            "qu_user": "$quoted_status.user.name",
            "qu_verified": "$quoted_status.user.verified"
        }
    },
    {
        "$group": {
            "_id": "$qu_user",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "count": -1
        }
    }
]

In [7]:
group_df = pd.DataFrame.from_dict(list(coll.aggregate(group_pipeline)))
group_df[:20]

,_id,count
0,Christi Belcourt,29
1,Indigenous Youth for Wet’suwet’en,28
2,Andrew Kurjata 📻,24
3,Gidimt’en Checkpoint,23
4,Torrance Coste,14
5,Russ Diabo,14
6,CBC News Alerts,13
7,Smogelgem,13
8,ChantelleBellrichard,10
9,Ricochet,10


In [8]:
# pivoting with $group:
# https://docs.mongodb.com/manual/reference/operator/aggregation/group/#group-documents-by-author

group_multilevel = [
    {
      "$match": {
          "is_quote_status": {"$eq": True},
          "quoted_status": {"$exists": True}
      }  
    },
    {
        "$project": {
            "_id": 0,
            "twid": "$id",
            "user_id": "$user.id",
            "user": "$user.name",
            "verified": "$user.verified",
            "created_at": {
                "$dateFromString": {
                    "dateString": "$created_at"
                }
            },
            "qu_twid": "$quoted_status.id_str",
            "qu_user_id": "$quoted_status.user.id_str",
            "qu_user": "$quoted_status.user.name",
            "qu_verified": "$quoted_status.user.verified"
        }
    },
    {
        "$group": {
            "_id": "$user_id",
            "tweet": {"$push": "$$ROOT"}
        }
    },
    {
        "$project": {
            "qu_user_id": "$tweet.qu_user_id",
            "qu_user": "$tweet.qu_user"
        }
    }
]

In [39]:
list(coll.aggregate(group_multilevel))[:10]

[{'_id': 896114542647795712,
  'qu_user_id': ['21166754', '834182874307768321'],
  'qu_user': ['Andrew Kurjata 📻', 'Scott Fraser']},
 {'_id': 14740104,
  'qu_user_id': ['1849355096'],
  'qu_user': ['Margareta Dovgal']},
 {'_id': 198263394,
  'qu_user_id': ['1082000847368679424',
   '1082000847368679424',
   '90790673',
   '86653536'],
  'qu_user': ['Gidimt’en Checkpoint',
   'Gidimt’en Checkpoint',
   'Geneviève Reynolds',
   'submedia']},
 {'_id': 818489629603221505,
  'qu_user_id': ['36395646'],
  'qu_user': ['Pam Palmater']},
 {'_id': 839225654172401664, 'qu_user_id': [], 'qu_user': []},
 {'_id': 2765828518,
  'qu_user_id': ['383304446', '177287178', '1082000847368679424'],
  'qu_user': ['Michael Toledano',
   'Morganne Campbell',
   'Gidimt’en Checkpoint']},
 {'_id': 15694414,
  'qu_user_id': ['274806875'],
  'qu_user': ['Kate Elizabeth Korte']},
 {'_id': 1165698855247241217,
  'qu_user_id': ['39446834'],
  'qu_user': ['Frank J Alec']},
 {'_id': 333573416, 'qu_user_id': [], 'qu_use

In [40]:
ml_df = pd.DataFrame.from_dict(list(coll.aggregate(group_multilevel)))
ml_df.to_json('quote_tweets_by_user.json', orient='records')

In [19]:
# pivoting with $group:
# https://docs.mongodb.com/manual/reference/operator/aggregation/group/#group-documents-by-author

# structuring json for d3 force-directed graphs:
# https://github.com/d3/d3-force

group_d3 = [
    {
      "$match": {
          "is_quote_status": {"$eq": True},
          "quoted_status": {"$exists": True}
      }  
    },
    {
        "$project": {
            "_id": 0,
            "source": "$user.id",
            "target": "$quoted_status.user.id",
        }
    }
]

In [33]:
links = pd.DataFrame.from_dict(list(coll.aggregate(group_d3)))
links.head()

,source,target
0,846507587457765376,17138582
1,956223400120168448,395335392
2,1221182879088037888,1006419421244678144
3,3193766737,3020467682
4,828104684397219840,935617448409206784


In [34]:
userlist = list(user_df['id'])

In [35]:
np.set_printoptions(suppress=True) # don't convert to scientific notation

def not_in_user_list(row: int) -> int:
    target = row['target']
    if target in userlist:
        return target
    print(target)
    return None

In [36]:
links['target_2'] = links.apply(not_in_user_list, axis=1)
links.head()

1101971425701715969
2690543658
325076723
245805420
228529107
134513525
242827267


,source,target,target_2
0,846507587457765376,17138582,1.713858e+07
1,956223400120168448,395335392,3.953354e+08
2,1221182879088037888,1006419421244678144,1.006419e+18
3,3193766737,3020467682,3.020468e+09
4,828104684397219840,935617448409206784,9.356174e+17


In [37]:
links.dtypes

source        int64
target        int64
target_2    float64
dtype: object

In [42]:
links[links.target_2.isna()]

,source,target,target_2
134,742354529086242816,1101971425701715969,NaN
833,2751081091,2690543658,NaN
836,25391299,325076723,NaN
840,27947724,245805420,NaN
842,1111459024589877248,228529107,NaN
847,572239210,134513525,NaN
850,295358506,242827267,NaN


In [43]:
to_delete = links[links.target_2.isna()]

In [44]:
to_delete.index

Int64Index([134, 833, 836, 840, 842, 847, 850], dtype='int64')

In [45]:
links.size

2553

In [47]:
links.drop(to_delete.index, inplace=True)

In [48]:
links.size

2532

In [50]:
links.drop(columns=['target_2'], inplace=True)
links.head()

,source,target
0,846507587457765376,17138582
1,956223400120168448,395335392
2,1221182879088037888,1006419421244678144
3,3193766737,3020467682
4,828104684397219840,935617448409206784


In [51]:
links.to_json('quote_links.json', orient='records')